## Part 1: Scrape Data and Clean Dataframe

In [6]:
###download/import dependencies###############################################################

##download packages if not already downloaded
import sys
!conda install --yes -c conda-forge geocoder
!conda install --yes -c conda-forge folium=0.5.0 
!conda install --yes -c conda-forge geopy

# import packages
import numpy as np 
import geocoder
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import pandas as pd
from unicodedata import normalize
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-64::anaconda==2020.02=py37_0
failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done
Solving environment: done

## Package Plan ##

  environment location: /Users/mckenziemandich/anaconda3

  added / updated specs:
    - geocoder


The following NEW packages will be INSTALLED:

  geocoder           conda-forge/noarch::geocoder-1.38.1-py_1
  ratelim            conda-forge/noarch::ratelim-0.1.6-py_2


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-64::anaconda==2020.02=py37_0
failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done
Solving environment: d

In [7]:
###download dataset###########################################################################

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#download all tables from wikipedia
wiki = pd.read_html(url, header = 0)

#get number of tables
print(f'Total tables: {len(wiki)}')

#assign df and check data
postcodes = wiki[0] #our table is the first one downloaded
postcodes = postcodes.rename(columns={"Neighbourhood":"neighbourhood"})
postcodes.head()


Total tables: 3


,Postal Code,Borough,neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
###clean dataset##############################################################################

#remove postcodes with borough listed as N/A
postcodes = postcodes[postcodes['Borough'] != 'Not assigned']

#reset index
postcodes.reset_index(drop = True).head()

#leave neighbourhoods as they are for now; will explode later

,Postal Code,Borough,neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
postcodes.shape

(103, 3)


## Part 2: Get Lat and Lon for neighbourhoods

In [10]:
###my attempt at using geocoder (doesn't work, geo.latlng returns none)#######################

# # import geocoder
# import geocoder 

# # loop until you get the coordinates
# for idx, row in postcodes.iterrows():
#     geo = geocoder.google('{}, Toronto, Ontario'.format(postcodes.iloc[0,0]))
#     lat_lng_coords = geo.latlng
#     postcodes.iloc[idx,3] = lat_lng_coords[0]
#     postcodes.iloc[idx,4] = lat_lng_coords[1]

In [11]:
###import lat/long by csv instead#############################################################

#import csv with lats/lons & check
coords = pd.read_csv("Geospatial_Coordinates.csv").rename(columns={"Neighbourhood":"neighbourhood"})
print(coords.shape) #check length matches cleaned dataset
coords.head()

#merge postcode and coordinate dfs
postcodes_coords = postcodes.merge(coords, how = 'left', on = ['Postal Code'])
postcodes_coords.head()

(103, 3)


,Postal Code,Borough,neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [12]:
###create map of toronto neighbourhoods using lat/lon#########################################

map_toronto = folium.Map(location=[43.7,-79.4], zoom_start=10)

# add markers to map
for lat, lng, label in zip(postcodes_coords['Latitude'], postcodes_coords['Longitude'], 
                           postcodes_coords['neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.8,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Part 3: Explore and Cluster neighbourhoods

In [13]:
###Save foursquare client ID and secret for calls#############################################
CLIENT_ID = 'UM5EZ1MMXBBAIUR2BERSL0ECQCUYANKC3ZBKWHCWDSXAUHXO'
CLIENT_SECRET = 'OGSUKEERLGXANFE2YXDUDLTLGQASHHAA5CISRCEVX3DFOBG4'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value


In [14]:
###make table of lat/lon for all neighbourhoods###############################################

#explode neighbourhoods with multiple comma-delimited entries
neighbourhoods = postcodes_coords[['neighbourhood','Latitude','Longitude']]

#create series from neighbourhood column split by ,
s = neighbourhoods['neighbourhood'].str.split(',').apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1) # to line up with df's index
s.name = 'neighbourhood' # needs a name to join
print(s.head(10)) #check indices

#remove old neighbourhoods column and join on index
neighbourhoods = neighbourhoods.drop(columns=['neighbourhood'])
neighbourhoods = neighbourhoods.join(s).reset_index(drop = True)
neighbourhoods.head()

0                         Parkwoods
1                  Victoria Village
2                       Regent Park
2                      Harbourfront
3                    Lawrence Manor
3                  Lawrence Heights
4                      Queen's Park
4     Ontario Provincial Government
5                  Islington Avenue
5             Humber Valley Village
Name: neighbourhood, dtype: object


,Latitude,Longitude,neighbourhood
0,43.753259,-79.329656,Parkwoods
1,43.725882,-79.315572,Victoria Village
2,43.654260,-79.360636,Regent Park
3,43.654260,-79.360636,Harbourfront
4,43.718518,-79.464763,Lawrence Manor


In [15]:
###function to get venues near a neighbourhood (from lab)####################################

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request (formatted to avoid key errors if groups not found)
        results = requests.get(url).json()
        results = results.get('response', {}).get('groups', {})
        results = results[0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['neighbourhood', 
                  'neighbourhood Latitude', 
                  'neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
####get venues for each neighbourhood#########################################################
# a bit slow

toronto_venues = getNearbyVenues(names=neighbourhoods['neighbourhood'],
                                   latitudes=neighbourhoods['Latitude'],
                                   longitudes=neighbourhoods['Longitude']
                                  )
print(toronto_venues.shape)
toronto_venues.head()

Parkwoods
Victoria Village
Regent Park
 Harbourfront
Lawrence Manor
 Lawrence Heights
Queen's Park
 Ontario Provincial Government
Islington Avenue
 Humber Valley Village
Malvern
 Rouge
Don Mills
Parkview Hill
 Woodbine Gardens
Garden District
 Ryerson
Glencairn
West Deane Park
 Princess Gardens
 Martin Grove
 Islington
 Cloverdale
Rouge Hill
 Port Union
 Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate
 Bloordale Gardens
 Old Burnhamthorpe
 Markland Wood
Guildwood
 Morningside
 West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
 Wilson Heights
 Downsview North
Thorncliffe Park
Richmond
 Adelaide
 King
Dufferin
 Dovercourt Village
Scarborough Village
Fairview
 Henry Farm
 Oriole
Northwood Park
 York University
East Toronto
 Broadview North (Old East York)
Harbourfront East
 Union Station
 Toronto Islands
Little Portugal
 Trinity
Kennedy Park
 Ionview
 East Birchmo

,neighbourhood,neighbourhood Latitude,neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [17]:
###one hot encoding for clustering############################################################
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['neighbourhood'] = toronto_venues['neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(4276, 269)


,neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [18]:
###group number of venue types by neighbourhood
toronto_grouped = toronto_onehot.groupby('neighbourhood').mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped.head()

(203, 269)


,neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.020833,0.0,0.0,0.010417,0.010417,0.0,0.010417,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,0.010417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [19]:
### get number of venues of each type for each neighbourhood and print

num_top_venues = 5

for hood in toronto_grouped['neighbourhood']: #slow
    temp = toronto_grouped[toronto_grouped['neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

            venue  freq
0     Coffee Shop  0.08
1            Café  0.05
2      Restaurant  0.04
3   Deli / Bodega  0.03
4  Clothing Store  0.03


                       venue  freq
0                       Park  0.33
1                 Playground  0.33
2               Intersection  0.33
3          Accessories Store  0.00
4  Middle Eastern Restaurant  0.00


                  venue  freq
0         Grocery Store  0.22
1              Pharmacy  0.11
2            Beer Store  0.11
3  Fast Food Restaurant  0.11
4        Sandwich Place  0.11


                 venue  freq
0      Airport Service  0.19
1       Airport Lounge  0.12
2          Coffee Shop  0.06
3  Rental Car Location  0.06
4   Airport Food Court  0.06


                  venue  freq
0         Grocery Store  0.22
1              Pharmacy  0.11
2            Beer Store  0.11
3  Fast Food Restaurant  0.11
4        Sandwich Place  0.11


          venue  freq
0      Pharmacy  0.14
1   Coffee Shop  0.14
2   Pizza Place  0.14
3  Liquor Stor

                        venue  freq
0                      Bakery  0.25
1                        Park  0.25
2  Construction & Landscaping  0.25
3            Basketball Court  0.25
4   Middle Eastern Restaurant  0.00


          venue  freq
0      Pharmacy  0.14
1   Coffee Shop  0.14
2   Pizza Place  0.14
3  Liquor Store  0.14
4    Beer Store  0.14


                             venue  freq
0                           Bakery   1.0
1                Accessories Store   0.0
2        Middle Eastern Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


                             venue  freq
0                             Park   0.5
1                   Sandwich Place   0.5
2                Accessories Store   0.0
3               Mexican Restaurant   0.0
4  Molecular Gastronomy Restaurant   0.0


                       venue  freq
0  Middle Eastern Restaurant  0.25
1              Shopping Mall  0.12
2      Vietnamese Restaurant  0.12
3           

                    venue  freq
0       Convenience Store  0.07
1  Thrift / Vintage Store  0.07
2            Burger Joint  0.07
3          Sandwich Place  0.07
4              Kids Store  0.07


                  venue  freq
0         Grocery Store  0.22
1              Pharmacy  0.11
2            Beer Store  0.11
3  Fast Food Restaurant  0.11
4        Sandwich Place  0.11


                 venue  freq
0          Coffee Shop  0.13
1             Aquarium  0.05
2                Hotel  0.04
3                 Café  0.04
4  Fried Chicken Joint  0.03


                           venue  freq
0                            Bar  0.12
1                    Coffee Shop  0.07
2                     Restaurant  0.05
3          Vietnamese Restaurant  0.05
4  Vegetarian / Vegan Restaurant  0.05


         venue  freq
0  Coffee Shop  0.11
1         Café  0.07
2        Hotel  0.05
3          Gym  0.04
4   Restaurant  0.04


                 venue  freq
0          Coffee Shop  0.13
1             Aquarium  0.

               venue  freq
0     Medical Center  0.12
1               Bank  0.12
2     Breakfast Spot  0.12
3       Intersection  0.12
4  Electronics Store  0.12


                 venue  freq
0          Coffee Shop  0.13
1             Aquarium  0.05
2                Hotel  0.04
3                 Café  0.04
4  Fried Chicken Joint  0.03


                       venue  freq
0            Thai Restaurant  0.08
1         Mexican Restaurant  0.08
2                       Café  0.08
3  Cajun / Creole Restaurant  0.04
4                     Bakery  0.04


                      venue  freq
0      Fast Food Restaurant   0.2
1  Mediterranean Restaurant   0.2
2                   Dog Run   0.2
3               Golf Course   0.2
4                      Pool   0.2


                    venue  freq
0             Pizza Place  0.33
1  Furniture / Home Store  0.33
2            Intersection  0.33
3                    Park  0.00
4                   Motel  0.00


                             venue  freq
0      

                       venue  freq
0          Convenience Store   1.0
1          Accessories Store   0.0
2  Middle Eastern Restaurant   0.0
3        Moroccan Restaurant   0.0
4        Monument / Landmark   0.0


                       venue  freq
0  Middle Eastern Restaurant  0.25
1              Shopping Mall  0.12
2      Vietnamese Restaurant  0.12
3                Auto Garage  0.12
4             Sandwich Place  0.12


              venue  freq
0  Ramen Restaurant  0.08
1       Pizza Place  0.08
2       Coffee Shop  0.08
3    Sandwich Place  0.05
4  Sushi Restaurant  0.05


                       venue  freq
0                Coffee Shop  0.67
1      Korean BBQ Restaurant  0.33
2          Accessories Store  0.00
3  Middle Eastern Restaurant  0.00
4        Monument / Landmark  0.00


          venue  freq
0  Skating Rink  0.25
1           Spa  0.12
2          Park  0.12
3      Bus Stop  0.12
4    Beer Store  0.12


                        venue  freq
0           Electronics Store  0.25


In [20]:
###function to get most common venues for each neighbourhood

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
### 

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['neighbourhood'] = toronto_grouped['neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Adelaide,Coffee Shop,Café,Restaurant,Thai Restaurant,Gym
1,Agincourt North,Park,Intersection,Playground,Dog Run,Dessert Shop
2,Albion Gardens,Grocery Store,Fried Chicken Joint,Beer Store,Sandwich Place,Discount Store
3,Bathurst Quay,Airport Service,Airport Lounge,Boat or Ferry,Plane,Coffee Shop
4,Beaumond Heights,Grocery Store,Fried Chicken Joint,Beer Store,Sandwich Place,Discount Store


# Clustering

In [22]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=int32)

In [23]:
neighbourhoods['neighbourhood'] = neighbourhoods['neighbourhood'].astype('object')
neighbourhoods_venues_sorted['neighbourhood'] = neighbourhoods_venues_sorted['neighbourhood'].astype('object')

In [24]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge venues with neighbourhoods to add latitude/longitude for each neighbourhood
toronto_merged = neighbourhoods.merge(neighbourhoods_venues_sorted, on='neighbourhood')

toronto_merged.head() # check the last columns!

,Latitude,Longitude,neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,43.753259,-79.329656,Parkwoods,0,Park,Food & Drink Shop,Construction & Landscaping,Yoga Studio,Doner Restaurant
1,43.725882,-79.315572,Victoria Village,0,Intersection,Hockey Arena,Coffee Shop,Pizza Place,Portuguese Restaurant
2,43.654260,-79.360636,Regent Park,0,Coffee Shop,Pub,Bakery,Park,Café
3,43.654260,-79.360636,Harbourfront,0,Coffee Shop,Pub,Bakery,Park,Café
4,43.718518,-79.464763,Lawrence Manor,0,Clothing Store,Furniture / Home Store,Accessories Store,Boutique,Event Space


In [25]:
###create map of clusters###################################################################

map_clusters = folium.Map(location=[43.7,-79.4], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)+1))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
   
    if cluster=='NaN':
        cluster=0
    else: 
        pass
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color= rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,-79.329656,Food & Drink Shop,Construction & Landscaping,Yoga Studio,Doner Restaurant
1,-79.315572,Hockey Arena,Coffee Shop,Pizza Place,Portuguese Restaurant
2,-79.360636,Pub,Bakery,Park,Café
3,-79.360636,Pub,Bakery,Park,Café
4,-79.464763,Furniture / Home Store,Accessories Store,Boutique,Event Space
5,-79.464763,Furniture / Home Store,Accessories Store,Boutique,Event Space
6,-79.389494,Sushi Restaurant,Gym,Nightclub,Beer Bar
7,-79.389494,Sushi Restaurant,Gym,Nightclub,Beer Bar
8,-79.194353,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner
9,-79.194353,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner


In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,-79.160497,Bar,Yoga Studio,Donut Shop,Diner
23,-79.160497,Bar,Yoga Studio,Donut Shop,Diner
24,-79.160497,Bar,Yoga Studio,Donut Shop,Diner
163,-79.394420,Airport Lounge,Boat or Ferry,Plane,Coffee Shop
164,-79.394420,Airport Lounge,Boat or Ferry,Plane,Coffee Shop
165,-79.394420,Airport Lounge,Boat or Ferry,Plane,Coffee Shop
166,-79.394420,Airport Lounge,Boat or Ferry,Plane,Coffee Shop
167,-79.394420,Airport Lounge,Boat or Ferry,Plane,Coffee Shop
168,-79.394420,Airport Lounge,Boat or Ferry,Plane,Coffee Shop
169,-79.394420,Airport Lounge,Boat or Ferry,Plane,Coffee Shop


In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
105,-79.532242,Yoga Studio,Donut Shop,Diner,Discount Store
106,-79.532242,Yoga Studio,Donut Shop,Diner,Discount Store
198,-79.498509,Construction & Landscaping,Baseball Field,Yoga Studio,Donut Shop
199,-79.498509,Construction & Landscaping,Baseball Field,Yoga Studio,Donut Shop
200,-79.498509,Construction & Landscaping,Baseball Field,Yoga Studio,Donut Shop
201,-79.498509,Construction & Landscaping,Baseball Field,Yoga Studio,Donut Shop
202,-79.498509,Construction & Landscaping,Baseball Field,Yoga Studio,Donut Shop
203,-79.498509,Construction & Landscaping,Baseball Field,Yoga Studio,Donut Shop
204,-79.498509,Construction & Landscaping,Baseball Field,Yoga Studio,Donut Shop
205,-79.498509,Construction & Landscaping,Baseball Field,Yoga Studio,Donut Shop


In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
192,-79.506944,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant
193,-79.506944,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant
194,-79.506944,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant


In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
103,-79.532242,Baseball Field,Yoga Studio,Eastern European Restaurant,Distribution Center
104,-79.532242,Baseball Field,Yoga Studio,Eastern European Restaurant,Distribution Center
196,-79.498509,Baseball Field,Yoga Studio,Drugstore,Distribution Center
197,-79.498509,Baseball Field,Yoga Studio,Drugstore,Distribution Center
198,-79.498509,Baseball Field,Yoga Studio,Drugstore,Distribution Center
199,-79.498509,Baseball Field,Yoga Studio,Drugstore,Distribution Center
200,-79.498509,Baseball Field,Yoga Studio,Drugstore,Distribution Center
201,-79.498509,Baseball Field,Yoga Studio,Drugstore,Distribution Center
202,-79.498509,Baseball Field,Yoga Studio,Drugstore,Distribution Center
203,-79.498509,Baseball Field,Yoga Studio,Drugstore,Distribution Center
